# Coursework 3: Bilinear Inverse Problems and Low-Rank Matrix Recovery

[] By tick the checkbox, we hereby declare that this coursework report is our own and autonomous work. We have acknowledged all material and sources used in its preparation, including books, articles, reports, lecture notes, internet software packages, and any other kind of document, electronic or personal communication. This work has not been submitted for any other assessment.

## 3.1 Test Data Generation (10%)

We consider the low-rank matrix completion problem given by 
$$
    \bm{y} = \mathcal{P}_{\Omega}(\bm{X}) 
$$
where $\bm{X} \in \mathbb{R}^{m \times n}$ is a low rank matrix of rank $r$. 

Data generation: Write $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$, where $\bm{U} \in \mathbb{R}^{m \times r}$, $\bm{G} \in \mathbb{R}^{r \times r}$, and $\bm{V} \in \mathbb{R}^{n \times r}$ are matrices with i.i.d. $\mathcal{N}(0,1)$ Gaussian entries. (Note that by $\bm{X} = \bm{U} \bm{G} \bm{V}^{\mathsf{T}}$ we are not talking about SVD.)

Design and implement a function `LRMC_data_gen` to generate test data. Provide necessary documentation.

In [6]:
using LinearAlgebra

function gen_GaussianMat(m,n)
  #normalize columns
  A = randn(m,n)
  for i = 1:n
    v = A[:,i]
    A[:,i] = v/norm(v,2)
  end
  return A
end

function LRMC_data_gen(r, m, n)
  G = Diagonal(randn(r))
  U = gen_GaussianMat(m, r)
  V = gen_GaussianMat(n, r)
  X = U*G*transpose(V)

  return X
end

test = LRMC_data_gen(2, 32, 48)
rank(test)
    

2

In [6]:
using LinearAlgebra

function gen_Gaussian_Mat(m::Int,n::Int)

    A = randn(m,n)
    for i = 1:n
        v = A[:,i]
        A[:,i] = v/norm(v,2)
    end
    return A
end

function generate_Omega(m::Int, n::Int, size_Omega::Int)
    Omega = zeros(Int,m,n)

    count = 0 
    while count != size_Omega 
        i = rand(1:m); j = rand(1:n)
        if Omega[i,j] != 1
            Omega[i,j] = 1
            count +=1 
        end
    end
    return Omega
end

function linear_projection(Omega::Matrix{Int64},X)
    y = []
    for i = 1:size(X,1) 
        for j = 1:size(X,2)
            if Omega[i,j] == 1 
                append!(y,X[i,j])
            end
        end        
    end
    return y
end

function LRMC_data_gen(r::Int,m::Int,n::Int, Omega::Matrix{Int64}) 
    G = Diagonal(randn(r)) 
    U = gen_Gaussian_Mat(m,r)
    V = gen_Gaussian_Mat(n,r)
    X = U*G*transpose(V)
    y = linear_projection(Omega,X)
    return y,X 
end
r = 2
m = 3; n = 8; size_Omega = Int(1/8 * m * n) ;

Omega = generate_Omega(m,n,size_Omega)

y, X = LRMC_data_gen(r,m,n,Omega)

X

3×8 Matrix{Float64}:
 -0.329252    0.138748    0.0519582    0.0348793  …   0.0619741   -0.056721
  0.117844   -0.0701854  -0.032908     0.038088      -0.0294626    0.109491
  0.0502099  -0.0159332  -0.00428003  -0.0181936     -0.00759719  -0.0140564

## 3.2 Matrix Completion Techniques

In the following, the suggested simulation setup is that $m = 32$, $n=48$, $r$ varies in $2:2:8$, and $|\Omega|/mn$ varies in $\{1/8,~ 1/6,~ 1/4,~ 1/2\}$. 

### 3.2.1 Alternating Minimization (20%)

Design, implement, and run tests for the alternating minimization method for low-rank matrix completion. Use the function name `LRMCRec_AM`. Provide necessary documentation.

### 3.2.2 Iterative Hard Thresholding (IHT) (20%)

Design, implement, and run simple tests for the IHT algorithm for low-rank matrix completion. Use the function name `LRMCRec_IHT`. Provide necessary documentation. 

### 3.2.3 Iterative Shrinkage-Thresholding Algorithm (ISTA) (25%)

Design, implement, and run simple tests for ISTA (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ISTA`. Provide necessary documentation. Use simulations to discuss the choice of parameters.

### 3.2.4 Lasso-ADMM (25%)

Design, implement, and run simple tests for an ADMM algorithm (to solve the Lasso formulation) for low-rank matrix completion. Use the function name `LRMCRec_ADMM`. Provide necessary documentation. Compare ADMM and ISTA in terms of convergence.

Given that the corresponding ADMM iterations are:
$$
x^{k+1} = arg min_x \frac{1}{2}||y - Ax||_{2}^{2} + \frac{\rho}{2}||x - z_{k} + \frac{v_{k}}{\rho}||_{2}^{2}
$$
$$
z^{k+1} = arg min_z \lambda ||z||_{1} + \frac{\rho}{2}||x_{k+1} - z + \frac{v_{k}}{\rho}||_{2}^{2}
$$
$$
v^{k+1} = v_{k} + \rho (x^{k+1} - z^{k+1})
$$

For $x^{k+1}$, we can calculate the closed form solutions by setting $\nabla f(x)=0$
$$
x = (A^{T}A + \rho)^{-1} (A^{T}y + \rho z^{k} - v^{k})
$$
We can do the same for $z^{k+1}$. We just need to be wary of the sign(z) that appears when differentiating the l1 norm.
$$
\lambda sign(z) + \rho z = \rho x^{k+1} + v^{k}
$$

When $z>0$ :
$$
\rho z = \rho x^{k+1} + v^{k} - \lambda
$$
$$
z = x^{k+1} + \frac{v^{k}}{\rho} - \frac{\lambda}{\rho}
$$

When $z<0$:
$$
z = x^{k+1} + \frac{v^{k}}{\rho} + \frac{\lambda}{\rho}
$$

In [ ]:
using LinearAlgebra
using SparseArrays

#=
For solving lasso problem with admm:

    0.5(y-Ax)^2 + lambda*norm(x,1)
    
=#
function LRMCRec_ADMM(y, A, lambda, rho)
    maxiter = 500
    loss = []
    rho = 1e-4 #set parameter

    c = size(y,2)
    r = size(A,2)

    I = sparse(I,r,r)
    #Initialise
    
    z = randn(r,c)
    v = zeros(size(y,1), size(y,2))

    for i in 1:maxiter
        x = inv(A'*A + rho * I) * (A'*y + rho * z - v)

        if z >= 0
            z = x + v/rho - lambda/rho
        end
        if z < 0
            z = x + v/rho + lambda/rho
        end
        v = v + rho * (x - z)
        
        cost = 0.5 * norm(y - A*x, 2) + lambda * norm(x, 1)
        append!(loss, cost)
    end
end
    

## Highlight

Please list a couple of highlights of your coursework that may impress your markers.